In [2]:
from qiskit import QuantumCircuit, execute, ClassicalRegister, QuantumRegister
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.ibmq import least_busy

from qiskit.circuit import classical_function, Int1, QuantumCircuit

import matplotlib.pyplot as plt
import numpy as np

from qiskit.visualization import plot_histogram

#### Apply __H__-gates to put qubits in superstate

In [3]:
def initialize_s(qc, qubits):
    for q in qubits:
        qc.h(q)
    return qc

#### Construct a diffuser for circuit of __n__ qubits

In [4]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

#### Build noise model from backend properties

In [5]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_lima')
noise_model = NoiseModel.from_backend(backend)

basis_gates = noise_model.basis_gates
coupling_map = backend.configuration().coupling_map


IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found.'

#### Construct the oracle with controlled Z-gates (XX110, XX101)

In [ ]:
n = 5
qc = QuantumCircuit(n)
qc.cz(0, 2)
qc.cz(1, 2)
oracle = qc.to_gate()
oracle.name = "U$_\omega$"

#### Consctruct the circuit

In [ ]:
qs = list(range(n))
grover_circuit = QuantumCircuit(n)
grover_circuit = initialize_s(grover_circuit, qs)
grover_circuit.append(oracle, qs)
grover_circuit.append(diffuser(n), qs)
grover_circuit.measure_all()
grover_circuit.draw()

#### Since there are 2 solutions and 8 possibilities, we only need to run one iteration

In [ ]:
# Perform a noise simulation
result = execute(grover_circuit, Aer.get_backend('aer_simulator'),
                 coupling_map=coupling_map,
                 basis_gates=basis_gates,
                 noise_model=noise_model).result()
counts = result.get_counts(0)
plot_histogram(counts)